In [1]:
import pandas as pd

# 读取文档内容
with open('LLM_benchmarks.md', 'r', encoding='utf-8') as file:
    content = file.read()

# 提取数据
data = []
lines = content.split('\n\n')
for line in lines:
    if line.startswith('##'):
        category = line[3:].strip()
    elif line.startswith('###'):
        subcategory = line[4:].strip()
    elif line.startswith('####'):
        model_name = line[5:].strip()
    elif line.startswith('URL'):
        url = line.split('URL')[1].strip()
        data.append([model_name, subcategory, category, url])

# 创建 DataFrame
df = pd.DataFrame(data, columns=['名称', '类型', '年份', '链接'])

# 写入 Excel 文件
df.to_excel('benchmark_data.xlsx', index=False)

处理OpenLLMs

In [10]:
import pandas as pd

# 读取Excel文件
df = pd.read_excel("ChinaLLMs.xlsx", sheet_name="OpenLLMs")

# 拆分参数规模列
df["参数规模（B）"] = df["参数规模"].str.replace("B", "").str.split(",")

# 将拆分后的列表展开成多行
df = df.explode("参数规模（B）")

# 重置索引
df.reset_index(drop=True, inplace=True)

# 删除原始的参数规模列
df.drop(columns=["参数规模"], inplace=True)

# 重命名列
df.rename(columns={"参数规模（B）": "参数规模"}, inplace=True)

# 将结果保存回原始Excel文件
with pd.ExcelWriter("ChinaLLMs.xlsx", engine="openpyxl", mode="a") as writer:
    df.to_excel(writer, sheet_name="OpenLLMs_split", index=False)


In [15]:
import pandas as pd

# 读取Excel文件
df = pd.read_excel("ChinaLLMs.xlsx", sheet_name="OpenLLMs")

# 创建一个字典来映射不同单位到字节的转换因子
unit_conversion = {"B": 1, "K": 1024, "M": 1024 ** 2, "G": 1024 ** 3, "T": 1024 ** 4}

# 定义一个函数来转换数据规模为字节
def convert_to_bytes(size):
    if pd.isna(size):  # 检查是否为空单元格
        return None
    unit = size[-1]  # 获取单位
    value = float(size[:-1])  # 获取数值部分并转换为浮点数
    return value * unit_conversion[unit]  # 返回转换后的字节数

# 将数据规模列应用转换函数
df["数据规模（B）"] = df["数据规模"].apply(convert_to_bytes)

# 删除原始的数据规模列
df.drop(columns=["数据规模"], inplace=True)

# 输出结果
print(df)

# 将结果保存回原始Excel文件
with pd.ExcelWriter("ChinaLLMs.xlsx", engine="openpyxl", mode="a") as writer:
    df.to_excel(writer, sheet_name="OpenLLMs_updated", index=False)


    序号                名称  参数规模（B）                                        说明  \
0    1           LLaMA-2      7.0                                       可商用   
1    1           LLaMA-2     13.0                                       可商用   
2    1           LLaMA-2     34.0                                       可商用   
3    1           LLaMA-2     70.0                                       可商用   
4    2            Falcon      7.0                            数据集 RefinedWeb   
..  ..               ...      ...                                       ...   
60  27            Yuan-2    102.0                                         源   
61  28  Mistral 7BX8 MoE     56.0                                  7BX8 MoE   
62  29            BlueLM      7.0     https://github.com/vivo-ai-lab/BlueLM   
63  30            YAYI 2     30.0   https://github.com/wenge-research/YAYI2   
64  31         CodeShell      7.0  https://github.com/WisdomShell/codeshell   

         数据规模（B）  
0   2.199023e+12  
1   2.199023e